# Exercise 1.2
Application for counting the number of cars that go from city downtown to city centre in peak hours.

We are using an algorithm based on frame differencing and background subtraction techniques like in the first task, so the code will be reused.
The main difference is the counting of the total number of cars and cars per minute.

In [83]:
import cv2
import time

source_file_1 = './media/Traffic_Laramie_1.mp4'
source_file_2 = './media/Traffic_Laramie_2.mp4'

In [84]:
def analyze_frame(
        videoSourceStr: str,
        setup_thresholds: bool = False,
    ) -> tuple:
    """
    Analyze the video frame by using a frame difference method.
    Returns a tuple with the total number of cars detected and cars per minute.
    """
    videoSource = cv2.VideoCapture(videoSourceStr)
    initial_frame = None
    # Threshold to determine if a pixel is different from the initial frame
    threshold = 10
    # Minimum area of a contour to be considered
    contour_area = 2500

    # Number of cars detected
    passing_cars = 0

    # Threshold time to count a car in seconds
    threshold_time = 1

    # Last time a car was detected
    last_time = time.time()

    # Total seconds of the video
    total_seconds = videoSource.get(cv2.CAP_PROP_FRAME_COUNT) / videoSource.get(cv2.CAP_PROP_FPS)

    if not setup_thresholds:
        print(f'Analyzing video source ${videoSourceStr} with {total_seconds} seconds')

    # Start playing video
    while True:
        ret, frame = videoSource.read()
        if not ret:
            if setup_thresholds:
                # Set the video on loop, this will allow us to continue setting the threshold
                videoSource.set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue
            else:
                break
        # We will create a ROI (Region of Interest) to focus on that area.
        height, width, _ = frame.shape
        half_height = int(height/2)
        half_width = int(width/2)
        roi = frame[half_height:int(height - half_height / 1.7), 0:half_width]

        # Background subtraction
        fgmask = cv2.createBackgroundSubtractorMOG2().apply(roi)
        fgmask = cv2.erode(fgmask, None, iterations=2)
        fgmask = cv2.dilate(fgmask, None, iterations=2)

        # Passing line coordinates
        passing_line = int(width/5)
        
        # Gray conversion and noise reduction
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (25,25), 0)

        # Save the first frame to compare with the next ones
        if initial_frame is None:
            initial_frame = blur
            continue
        
        # Get the absolute difference between the first frame and the current one
        delta_frame = cv2.absdiff(initial_frame, blur)
        threshold_frame = cv2.threshold(delta_frame, threshold, 255, cv2.THRESH_BINARY)[1]

        cv2.line(frame, (passing_line, half_height + 20), (passing_line, half_height + 150), (0,0,255), 2)

        combined_mask = cv2.bitwise_and(fgmask, threshold_frame)

        # Get contours of the threshold frame
        contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw rectangles around the contours
        for contour in contours:
            if cv2.contourArea(contour) < contour_area:
                continue
            x, y, w, h = cv2.boundingRect(contour)
            y = y + int(height/2)
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 1)

            ## If the contour crosses the line, count the car
            if passing_line > x and passing_line < x+w:
                if time.time() - last_time > threshold_time:
                    last_time = time.time()
                    cv2.line(frame, (passing_line, half_height + 20), (passing_line, half_height + 150), (0,255,0), 2)
                    cv2.putText(frame, 'Car detected', (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
                    passing_cars += 1
        
        if setup_thresholds:
            cv2.putText(frame, f'Threshold: {threshold}', (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            cv2.putText(frame, f'Contour Area: {contour_area}', (10,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            cv2.putText(frame, f'Passing cars: {passing_cars}', (10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

            cv2.imshow('frame', frame)

        key = cv2.waitKey(1)

        ## Set up control keys
        if key == ord('q'):
            break

        if setup_thresholds:
            # If setting up trhesholds, w and s keys will increase and decrease the threshold value. a and d keys will do the same for the contour area
            if key == ord('w'):
                threshold += 10
            if key == ord('s'):
                threshold -= 10
            if key == ord('a'):
                contour_area -= 100
            if key == ord('d'):
                contour_area += 100

    
    videoSource.release()
    cv2.destroyAllWindows()
    return passing_cars, passing_cars / total_seconds * 60

In [85]:
video_1_cars, video_1_cars_per_minute = analyze_frame(source_file_1)
video_2_cars, video_2_cars_per_minute = analyze_frame(source_file_2)
print(f'Video 1 cars: {video_1_cars}, cars per minute: {round(video_1_cars_per_minute, 2)}')
print(f'Video 2 cars: {video_2_cars}, cars per minute: {round(video_2_cars_per_minute, 2)}')

Analyzing video source $./media/Traffic_Laramie_1.mp4 with 177.92 seconds
Analyzing video source $./media/Traffic_Laramie_2.mp4 with 105.68 seconds
Video 1 cars: 6, cars per minute: 2.02
Video 2 cars: 4, cars per minute: 2.27
